<a href="https://colab.research.google.com/github/aphennGC/explore-assistant-model/blob/main/2_Explore_Assistant_Demo_(Make_a_copy_first).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explore Assistant
This document provides instructions on how to install and customize the Explore Assistant demo. The demo is a web application that allows users to explore data in Looker by asking questions in natural language.

DISCLAIMER : DO NOT INTENDED TO BE USED IN A PRODUCTION ENVIRONMENT

In [ ]:
# @title Setup : Installing Required Librairies
#!pip install looker_sdk
!pip install google-cloud-aiplatform --upgrade --user
!pip install pandas
!pip install -q gradio

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Authenticating your notebook environment

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

### Import libraries


Update Project and Region of your Vertex AI Project.

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel

PROJECT_ID = "project"  # @param {type:"string"}
LOCATION = "region" # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
from google.cloud import aiplatform
import gradio as gr
import urllib.parse
import json

# Print LangChain and Vertex AI versions
print(f"Gradio version: {gr.__version__}")
print(f"Vertex AI SDK version: {aiplatform.__version__}")

Gradio version: 4.12.0
Vertex AI SDK version: 1.38.1


### Initiating Vertex AI

In [ ]:
# Definig parameters to be used.
parameters = {
    "candidate_count": 1,
    "temperature": 0.2,
    "max_output_tokens": 100,
    "top_p": 0.8,
    "top_k": 40
}

model = TextGenerationModel.from_pretrained("text-bison")

### Looker's Semantic Context and Examples
Using the semantic layer as context for the LLM to extract the relevant field to build the query.

In [ ]:
# Default Base for Embedding the demo
looker_base_url = "https://ceworkshops.cloud.looker.com/embed/explore/thelookai/order_items?"

# Semantic Model Context : Result of the first Colab
context = """You\'re a developer who would transalate questions to a structured URL query based on the following dicitonnary - choose only the fileds in the below description

---- Past you Model Context Here
---- Paste your Examples here
"""

### Gradio's Functions
Function that would be used later by Gradio interface in order to render the results from Looker.

In [ ]:
# Savings Logs to further train the model.
def save_logs(input_text, output_text, filename):
    formatted_input_text = json.dumps(input_text)[1:-1]
    formatted_output_text = json.dumps(output_text)[1:-1]

    with open(filename, 'a') as file:
        line = f'{{"input_text": "{formatted_input_text}", "output_text": "{formatted_output_text}"}}\n'
        file.write(line)

    return True

# Define a function to process the search query and return results
def search(query):
    # Formatting prompt
    llm = """
    input: {}
    output: """.format(query)
    # Building the prompt
    predict = context + llm
    # Retrieving Results from Model
    response =  model.predict(predict,**parameters).text
    response = response.strip()
    # Saving Logs
    if save_logs(query, response, "logs.jsonl"):
            print("Logs Saved Correctly")
    # Percent Encoding
    query_url = urllib.parse.quote(response, safe=':/?&=,[]{}')
    # Building the Embedded URL
    url = f"{looker_base_url}{query_url}&toggle=dat,pik,vis&allow_login_screen=true"
    print(url)
    return f'<iframe src="{url}" style="width:100%; height:600px;"></iframe>'

## Running the demo

In [ ]:
# Create the Gradio app with custom layout
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
    """
    # Explore Assistant

    """)
    with gr.Row():
        with gr.Column(scale=10) :
            search_input = gr.Textbox(label="Type your question and press Enter", lines=1, placeholder="Sales this week ?")
    with gr.Row():
        search_results = gr.HTML()
        search_input.submit(search, inputs=search_input, outputs=search_results)

if __name__ == "__main__":
    demo.launch(debug=True)

## Showing Logs Saved

In [ ]:
# Define the path to your logs.jsonl file
jsonl_file_path = 'logs.jsonl'

# Read the JSON Lines file and process each line
with open(jsonl_file_path, 'r') as file:
    for line in file:
        # Load the line as a JSON object
        json_data = json.loads(line)

        # Extract the input and output strings
        input_string = json_data.get('input_text', '')
        output_string = json_data.get('output_text', '')

        # Format and print the data
        print(f'input: {input_string}')
        print(f'output: {output_string}')
        print()